In [1]:
import netCDF4
import pandas as pd
from netCDF4 import num2date
import numpy as np
from datetime import timedelta

In [2]:
# 文件路径
nc_file_path = 'E:\\DownLoad\\TotalP\\02.nc'
excel_file_path = 'E:\\DownLoad\\TotalP\\output1.xlsx'

In [3]:
# 经度和纬度范围
lon_min, lon_max = 45, 46  # 经度范围
lat_min, lat_max = 59, 60    # 纬度范围

In [4]:
# 打开nc文件
dataset = netCDF4.Dataset(nc_file_path)

# 获取时间、经度和纬度变量
time_var = dataset.variables['time']
time = time_var[:]
longitude = dataset.variables['longitude'][:]
latitude = dataset.variables['latitude'][:]
tp = dataset.variables['tp'][:]  # (time, latitude, longitude)

# 获取时间单位
time_units = time_var.units

# 将时间变量转换为 UTC 时间
time_dates_utc = num2date(time, units=time_units, calendar='gregorian')

# 定义时间偏移量
time_offset = timedelta(hours=8)  # UTC+8

In [5]:
# 找到经纬度范围内的索引
lon_indices = np.where((longitude >= lon_min) & (longitude <= lon_max))[0]
lat_indices = np.where((latitude >= lat_min) & (latitude <= lat_max))[0]

# 筛选经纬度范围内的数据
tp_filtered = tp[:, lat_indices, :][:, :, lon_indices]
filtered_latitude = latitude[lat_indices]
filtered_longitude = longitude[lon_indices]

In [6]:
# 创建一个ExcelWriter对象
with pd.ExcelWriter(excel_file_path, engine='openpyxl') as writer:
    for t_index, utc_date in enumerate(time_dates_utc):
        # 转换为 UTC+8 时间
        local_date = utc_date + time_offset

        # 获取当前时间步的数据
        tp_data = tp_filtered[t_index, :, :]

        # 创建DataFrame
        df = pd.DataFrame(tp_data, index=filtered_latitude, columns=filtered_longitude)
        
        # 创建工作表名称
        sheet_name = local_date.strftime('%Y-%m-%d_%H-%M-%S')  # 格式化为字符串
        
        # 将DataFrame写入不同的工作表
        df.to_excel(writer, sheet_name=sheet_name)

print(f'数据已成功保存到 {excel_file_path}')

数据已成功保存到 E:\DownLoad\TotalP\output1.xlsx
